# SQL 쿼리 생성기: 다중 노드 그래프

이 노트북에서는 **여러 노드를 연결**하여 **SQL 쿼리를 생성하고 설명**하는 시스템을 만듭니다.

## 이번에 만들 시스템

```
┌────────────────────────────────────────────────────────────────────┐
│                SQL 쿼리 생성기 아키텍처                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   사용자 입력: "각 품목의 판매량을 구해주세요"                       │
│        │                                                          │
│        ▼                                                          │
│   ┌─────────────────┐                                              │
│   │  generate_sql   │  ← 노드 1: SQL 쿼리 생성                     │
│   │  (temperature   │    정확한 SQL을 위해 낮은 temperature        │
│   │   = 0.1)        │                                              │
│   └────────┬────────┘                                              │
│            │                                                       │
│            ▼                                                       │
│   ┌─────────────────┐                                              │
│   │   explain_sql   │  ← 노드 2: SQL 설명 생성                     │
│   │  (temperature   │    자연스러운 설명을 위해 높은 temperature    │
│   │   = 0.7)        │                                              │
│   └────────┬────────┘                                              │
│            │                                                       │
│            ▼                                                       │
│   출력: {sql_query: "SELECT...", sql_explanation: "이 쿼리는..."}  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## Temperature란?

```
┌────────────────────────────────────────────────────────────────────┐
│                 Temperature의 역할                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   Temperature = 창의성/무작위성 조절 파라미터                       │
│                                                                    │
│   낮은 Temperature (0.1):                                          │
│   • 일관되고 예측 가능한 출력                                      │
│   • SQL 쿼리, 코드 생성에 적합                                     │
│   • "정답"이 있는 작업에 사용                                      │
│                                                                    │
│   높은 Temperature (0.7~1.0):                                      │
│   • 다양하고 창의적인 출력                                         │
│   • 자연어 설명, 글쓰기에 적합                                     │
│   • 다양성이 필요한 작업에 사용                                    │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 두 가지 LLM 설정

각 노드의 목적에 맞게 다른 temperature를 사용합니다.

In [ ]:
from langchain_ollama import ChatOllama

# SQL 쿼리 생성용 (정확성 중요 → 낮은 temperature)
model_low_temp = ChatOllama(model='llama3.2', temperature=0.1)

# 자연어 설명 생성용 (자연스러움 중요 → 높은 temperature)
model_high_temp = ChatOllama(model='llama3.2', temperature=0.7)

print("✅ 두 가지 LLM 설정 완료")
print("   - model_low_temp (temperature=0.1): SQL 생성용")
print("   - model_high_temp (temperature=0.7): 설명 생성용")

# 3. State 정의 (확장된 버전)

이번에는 더 많은 데이터를 저장합니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    SQL 생성기의 상태
    
    messages: 대화 기록
    user_query: 사용자의 원본 질문
    sql_query: 생성된 SQL 쿼리
    sql_explanation: SQL에 대한 설명
    """
    messages: Annotated[list, add_messages]  # 대화 기록
    user_query: str                          # 입력
    sql_query: str                           # 출력 1
    sql_explanation: str                     # 출력 2

# 입력과 출력 타입 정의 (선택 사항이지만 명확성을 위해)
class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    sql_query: str
    sql_explanation: str

print("✅ State, Input, Output 정의 완료")

# 4. 노드 1: SQL 쿼리 생성

사용자의 자연어 질문을 SQL 쿼리로 변환합니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# SQL 생성용 시스템 프롬프트
generate_prompt = SystemMessage(
    content='당신은 친절한 데이터 분석가입니다. 사용자의 질문을 바탕으로 SQL 쿼리를 작성하세요.'
)

def generate_sql(state: State) -> State:
    """
    노드 1: SQL 쿼리 생성
    
    1. 사용자 질문을 HumanMessage로 변환
    2. 시스템 프롬프트 + 대화 기록 + 사용자 질문 조합
    3. LLM으로 SQL 쿼리 생성
    4. 결과를 State에 저장
    """
    user_message = HumanMessage(content=state['user_query'])
    messages = [generate_prompt, *state['messages'], user_message]
    
    # 낮은 temperature로 정확한 SQL 생성
    res = model_low_temp.invoke(messages)
    
    return {
        'sql_query': res.content,
        'messages': [user_message, res],  # 대화 기록 업데이트
    }

print("✅ generate_sql 노드 정의 완료")

# 5. 노드 2: SQL 설명 생성

생성된 SQL 쿼리를 사용자가 이해하기 쉽게 설명합니다.

In [ ]:
# SQL 설명용 시스템 프롬프트
explain_prompt = SystemMessage(
    content='당신은 친절한 데이터 분석가입니다. 사용자에게 SQL 쿼리를 설명하세요.'
)

def explain_sql(state: State) -> State:
    """
    노드 2: SQL 쿼리 설명
    
    1. 이전 대화 기록 (사용자 질문 + SQL 쿼리) 가져오기
    2. LLM으로 자연스러운 설명 생성
    3. 결과를 State에 저장
    """
    messages = [
        explain_prompt,
        *state['messages'],  # 이전 단계의 사용자 질문과 SQL 쿼리
    ]
    
    # 높은 temperature로 자연스러운 설명 생성
    res = model_high_temp.invoke(messages)
    
    return {
        'sql_explanation': res.content,
        'messages': res,  # 대화 기록 업데이트
    }

print("✅ explain_sql 노드 정의 완료")

# 6. 그래프 구성

두 노드를 순차적으로 연결합니다.

```
START → generate_sql → explain_sql → END
```

In [ ]:
from langgraph.graph import StateGraph, START, END

# 그래프 빌더 생성 (입력/출력 타입 지정)
builder = StateGraph(State, input=Input, output=Output)

# 노드 추가
builder.add_node('generate_sql', generate_sql)
builder.add_node('explain_sql', explain_sql)

# 엣지 추가 (순차 실행)
builder.add_edge(START, 'generate_sql')       # 시작 → SQL 생성
builder.add_edge('generate_sql', 'explain_sql')  # SQL 생성 → SQL 설명
builder.add_edge('explain_sql', END)          # SQL 설명 → 종료

# 그래프 컴파일
graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. SQL 생성기 실행

In [ ]:
# 테스트 실행
result = graph.invoke({'user_query': '각 품목의 판매량을 구해주세요.'})

print("=== SQL 생성기 결과 ===")
print(f"\n📝 사용자 질문: 각 품목의 판매량을 구해주세요.")
print(f"\n💻 생성된 SQL:\n{result['sql_query']}")
print(f"\n📖 SQL 설명:\n{result['sql_explanation']}")

# 8. 다양한 질문으로 테스트

In [ ]:
test_queries = [
    "가장 많이 팔린 상품 3개를 알려주세요.",
    "월별 매출 합계를 구해주세요.",
    "고객별 구매 횟수를 계산해주세요.",
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"📝 질문: {query}")
    print("="*60)
    
    result = graph.invoke({'user_query': query})
    
    print(f"\n💻 SQL:\n{result['sql_query'][:200]}...")
    print(f"\n📖 설명:\n{result['sql_explanation'][:200]}...")

---

## 정리: 다중 노드 그래프

### 아키텍처

```
┌─────────────────────────────────────────────────────────────────────┐
│                    다중 노드 그래프 패턴                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   START → 노드1 → 노드2 → ... → 노드N → END                        │
│                                                                     │
│   각 노드는:                                                        │
│   • State를 입력으로 받음                                           │
│   • 자신의 작업을 수행                                              │
│   • State를 업데이트하여 반환                                       │
│   • 다음 노드가 업데이트된 State를 사용                             │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 확장된 State 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_query: str
    sql_query: str
    sql_explanation: str

# 2. 여러 노드 정의
def node1(state): ...
def node2(state): ...

# 3. 순차 연결
builder.add_edge(START, 'node1')
builder.add_edge('node1', 'node2')
builder.add_edge('node2', END)
```

### Temperature 선택 가이드

| 작업 유형 | Temperature | 이유 |
|----------|-------------|------|
| SQL/코드 생성 | 0.0 ~ 0.3 | 정확성 필요 |
| 분류/추출 | 0.0 ~ 0.3 | 일관성 필요 |
| 자연어 설명 | 0.5 ~ 0.8 | 자연스러움 필요 |
| 창의적 글쓰기 | 0.8 ~ 1.0 | 다양성 필요 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model_low_temp = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)
model_high_temp = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)

# 변경
model_low_temp = ChatOllama(model='llama3.2', temperature=0.1)
model_high_temp = ChatOllama(model='llama3.2', temperature=0.7)
```

## 다음 단계

**조건부 분기(Conditional Edge)**를 사용하여 도메인에 따라 다른 처리를 하는 **Multi-RAG** 시스템을 만듭니다. (06-07번 노트북)